In [1]:
url = 'https://www.rottentomatoes.com/browse/movies_at_home/genres:horror?page='
num_pages = 15

In [2]:
import requests 
from bs4 import BeautifulSoup 

In [3]:
import pandas as pd

class RottenTomatoesScraper(): 
    def __init__(self,url, num_pages): 
        self.url = url + str(num_pages)
        self.num_pages = num_pages
        self.page = BeautifulSoup(requests.get(self.url).text, "html.parser")
        self.movie_links = []
        self.movies_dataframe = pd.DataFrame()
        for i in self.page.find_all(attrs={'class':'js-tile-link'}):
            try: 
                self.movie_links.append('https://www.rottentomatoes.com' + str(i.attrs['href']))
            except: 
                pass 
    
    def scrape_movie_data(self):
        
        for movie_link in self.movie_links:
            extra_movie_dict = {}
            movie = BeautifulSoup(requests.get(movie_link).text, "html.parser")
            extra_movie_info = movie.find_all(attrs={ 'class':'panel-body content_body'})[0]
            extra_movie_list = extra_movie_info.text.replace('  ', '').replace('\n\n\n', '\n').split('\n')
            extra_movie_list = [item for item in extra_movie_list if item != '']
            extra_movie_dict['description'] = extra_movie_list[0] 
            cleaned_movie_list = []
            
            for i in range(1,len(extra_movie_list)):
                if (':' in extra_movie_list[i]) and ('.' not in extra_movie_list[i]): 
                    pass
                else: 
                    extra_movie_list[i-1] = extra_movie_list[i-1] + ' ' + extra_movie_list[i]
                    cleaned_movie_list.append(extra_movie_list[i-1])

            
            for item in cleaned_movie_list: 
                if (':' in item) and ('.' not in item): 
                    key_value_pair = item.split(':')
                    extra_movie_dict[key_value_pair[0].strip()] = key_value_pair[1].strip()
                elif (':' in item) and ('.' in item):
                    idx = item.index(':')
                    extra_movie_dict[item[:idx]] = item[idx + 1:]
        
            self.movies_dataframe = pd.concat([self.movies_dataframe, pd.DataFrame(extra_movie_dict,index = [0])], axis = 0)
        
            
            

    def scrape_movie_scores(self):
        elements = [item for item in self.page.find_all(attrs={'data-qa':'discovery-media-list-item-caption'})]
        thumbnail_images = [item for item in self.page.find_all(attrs={'class':'posterImage'})]
        movie_title = []
        movie_release_date = []
        critics_score = [] 
        audience_score = []
        critics_sentiment = [] 
        audience_sentiment = [] 
        movie_images = []
        
        for item in elements:
            try:
                scores = item.find_all('score-pairs')[0]
                critics_score.append(scores.attrs['criticsscore'])
                audience_score.append(scores.attrs['audiencescore'])
                critics_sentiment.append(scores.attrs['criticssentiment'])
                audience_sentiment.append(scores.attrs['audiencesentiment'])
                movie_title.append(item.text.split('\r\n        \n\r\n            ')[0].replace('\n\n\n\r\n            ',''))
                movie_release_date.append(item.text.split('\r\n        \n\r\n            ')[1].replace(' \r\n          \n',''))
            except:
                pass
        self.movies_dataframe['movie_title'] = movie_title 
        self.movies_dataframe['movie_release_date'] = movie_release_date
        self.movies_dataframe['critics_score'] = critics_score
        self.movies_dataframe['audience_score'] = audience_score
        self.movies_dataframe['critics_sentiment'] = critics_sentiment
        self.movies_dataframe['audience_sentiment'] = audience_sentiment
        
        for item in thumbnail_images:
            try:
                movie_images.append(item.attrs['src'])
            except:
                pass
        self.movies_dataframe['movie_image'] = movie_images 
        self.movies_dataframe = self.movies_dataframe[list(self.movies_dataframe.columns[-7:]) + list(self.movies_dataframe.columns[:-7])]
        self.movies_dataframe.reset_index(drop = True, inplace = True)

        
    
        

        

In [4]:
scraper_object = RottenTomatoesScraper('https://www.rottentomatoes.com/browse/movies_at_home/genres:horror?page=', num_pages)      

In [5]:
scraper_object.scrape_movie_data()
scraper_object.scrape_movie_scores()

In [6]:
scraper_object.movies_dataframe

,movie_title,movie_release_date,critics_score,audience_score,critics_sentiment,audience_sentiment,movie_image,description,Genre,Original Language,...,Writer,Release Date (Theaters),Release Date (Streaming),Runtime,Distributor,Rating,Box Office (Gross USA),Sound Mix,Aspect Ratio,View the collection
0,Blood,"Streaming Jan 31, 2023",55,75,negative,positive,https://resizing.flixster.com/vicviLC49CVUNdYS...,"Jess (Michelle Monaghan), a nurse and mother r...","Horror,",English,...,Will Honley,"Jan 27, 2023","Jan 31, 2023",1h 48m,Vertical Entertainment,NaN,NaN,NaN,NaN,NaN
1,Heretic,"Streaming Jan 31, 2023",,,,,https://resizing.flixster.com/BlBPEe1VhBAm5yiW...,"Hannah Blair tells people stories, predicts th...",Horror,English,...,Chris Schwab,"Jan 20, 2023","Jan 31, 2023",1h 27m,Indican Pictures,NaN,NaN,NaN,NaN,NaN
2,Alive,"Streaming Jan 31, 2023",,,,,https://resizing.flixster.com/OGht6agq8wG0iaAJ...,Helen navigates a ravaged world with her boyfr...,"Horror,",English (United Kingdom),...,"David Marantz,",NaN,"Jan 31, 2023",1h 33m,NaN,NaN,NaN,NaN,NaN,NaN
3,Viking Wolf,"Streaming Feb 3, 2023",,,,,https://resizing.flixster.com/kr1vJj96gOTQXTN1...,After witnessing a grotesque murder at a party...,"Horror,",Norwegian,...,"Espen Aukan,",NaN,"Feb 3, 2023",1h 37m,NaN,NaN,NaN,NaN,NaN,NaN
4,Arthur Malediction,"Streaming Feb 3, 2023",,,,,https://resizing.flixster.com/TGMn0DSHkoTX0QGk...,"Since childhood, Alex has been a big fan of th...",Horror,French (France),...,Luc Besson,NaN,"Feb 3, 2023",1h 27m,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Lullaby,"Streaming Dec 16, 2022",,46,,negative,https://resizing.flixster.com/GLtso6H_-0plTM1E...,The film follows a new mother who discovers a ...,Horror,English,...,"Alex Greenfield,","Dec 16, 2022","Dec 16, 2022",1h 29m,Vertical Entertainment,NaN,NaN,NaN,NaN,NaN
146,Kids vs. Aliens,"Streaming Jan 20, 2023",57,67,negative,positive,https://resizing.flixster.com/ljcR-tRstatbELCe...,All Gary wants is to make awesome home movies ...,"Holiday,",English,...,"Jason Eisener,","Jan 20, 2023","Jan 20, 2023",1h 15m,RLJE Films,NaN,NaN,NaN,NaN,NaN
147,Saw,"Streaming Oct 28, 2014",50,84,negative,positive,https://resizing.flixster.com/fzM522qktf3aPINn...,Photographer Adam Stanheight (Leigh Whannell) ...,"Horror,",English,...,"Leigh Whannell,","Oct 29, 2004","Feb 15, 2005",1h 43m,Lionsgate Films,R (Strong Grisly Violence|Language),$650.1K,"DTS, Dolby SRD, Dolby, Surround, Dolby Atmos",Flat (1.85:1),Saw
148,Titane,"Streaming Oct 19, 2021",90,85,positive,positive,https://resizing.flixster.com/nBdklfZXU1hAGFRd...,Titane: A metal highly resistant to heat and c...,"Horror,",French (France),...,Julia Ducournau,"Oct 1, 2021","Oct 19, 2021",1h 48m,Neon,R (Graphic Nudity|Disturbing Material|Language...,$1.3M,NaN,Scope (2.35:1),NaN


In [60]:
scraper_object.movies_dataframe.to_csv("rotten_tomatoes_horror_movies.csv")